<a href="https://colab.research.google.com/github/VielF/ColabProjects/blob/main/Segmenta%C3%A7%C3%A3o_por_Crescimento_de_Regi%C3%B5es_Parte_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Universidade do Vale do Itajaí<br>
Escola Politécnica<br>
Processamento Digital de Imagens

# Exercício Segmentação Clássica - Sem Deep Learning

### Tutoriais da OpenCV

- https://docs.opencv.org/master/d9/df8/tutorial_root.html
- https://www.geeksforgeeks.org/opencv-python-tutorial/

# Segmentação por Crescimento de Regiões - Parte 1

O sonho de consumo de todo aquele que desenvolve métodos de processamento de imagens é um algoritmo genérico para dividir a imagem em seus componentes. Para conseguir fazer isso, os algoritmos clássicos de segmentação baseiam-se em informação de semelhança entre os pixels e de continuidade de valores de pixels dentro da imagem (veja a minha digressão sobre *apriorismos Kantianos* na vídeoaula inicial onde eu apresento os domínios e as etapas da visão computacional), sendo provavelmente os algoritmos que de forma mais icônica incorporam o conceito de "domínio do espaço".

Infelizmente nenhum dos algoritmos clássicos de segmentação realiza esse sonho de consumo: um algoritmo clássico que se baseia unicamente na informação contida dentro da imagem necessariamente é um algoritmo que usa apenas informação de natureza *sintática*. Toda a informação que está à disposição do algoritmo de segmentação diz respeito apenas à própria natureza da *estrutura* (sintaxe) dos pixels: seus valores, suas posições relativas uns aos outros e seus padrões de variação no espaço. Como nenhum desses algoritmos incorpora a informação *semântica*, do significado de padrões de variação e organização, todos eles necessariamente vão dividir a imagem em regiões "parecidas", ignorando o fato de que um carro vermelho possui pneus pretos e faróis e para-choque prateados… Durante 50 anos da história da visão computacional, o pesadelo recorrente do desenvolvedor de algoritmos de interpretação de imagens foi o de juntar estes segmentos contendo paralama, capô, pneu, calota, para-choque, etc para formar uma imagem de um "automóvel". Na visão clássica isso só é possível de se fazer possuindo-se um *modelo* de qual é a aparência desse "automóvel". Essa informação, de caráter semântico, em muito extrapola o que um algoritmo de segmentação consegue fazer.

Tentativas de resolver esse problema através de um pós-processamento utilizando-se técnicas da Inteligência Artificial Simbólica Clássica Baseada em Modelos (*Model Based Reasoning*) fracassaram para todos os domínios de aplicação que fugiam do trivialmente simples. A visão computacional só conseguiu vencer essa barreira com o advento da Segmentação Semântica utilizando Redes Neurais de Aprendizado Profundo nos últimos anos. Vamos ver isso quando chegarmos nesse capítulo. Por enquanto, divirta-se com os algoritmos clássicos que foram o estado da arte durante quase 50 anos da história da visão computacional…


### Inicialização

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## K-Means

O algoritmo tradicional de análise de cluster aplicado à segmentação de imagens: pixels semelhantes que estão em regiões vizinhas são agrupados em regiões.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, interactive, interact_manual
from skimage.data import astronaut
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float
import cv2

image = cv2.imread("/content/ct-02.jpg", cv2.IMREAD_GRAYSCALE)
#image = img_as_float(img[::2, ::2])

# Este é uma variação do clássico algoritmo k-Means chamado SLIC. Leia sobre a diferença em
# https://core.ac.uk/reader/95407090
def my_k_means(n_segments, compactness, sigma, colormap):
    global image
    colormap = eval('plt.cm.' + colormap)
    segments_slic = slic(image, n_segments=n_segments, compactness=compactness, sigma=sigma, channel_axis=None)

    fig, ax = plt.subplots(ncols=2, figsize=(15, 7), sharex=True, sharey=True)

    ax[0].imshow(mark_boundaries(image, segments_slic))
    ax[0].set_title('Original com limites: SLIC (k-Means)')

    ax[1].imshow(segments_slic, cmap=colormap, interpolation='nearest')
    ax[1].set_title('Segmentos: SLIC (k-Means)')

    for a in ax:
        a.set_axis_off()


    plt.tight_layout()
    plt.show()

interactive(my_k_means, n_segments=(5, 100), compactness=(0.1, 1.0), sigma=(0.1, 3.0), colormap = ['nipy_spectral', 'hot', 'magma', 'seismic'])

interactive(children=(IntSlider(value=52, description='n_segments', min=5), FloatSlider(value=0.55, descriptio…

## Watershed

O Watershed (divisor de águas ou, em uma tradução literal, bacia hidrográfica) é um algoritmo clássico utilizado para segmentação, ou seja, para separar diferentes objetos em uma imagem.

A partir de marcadores definidos pelo usuário, o algoritmo Watershed trata os valores dos pixels como uma topografia local (elevação). O algoritmo inunda bacias a partir dos marcadores, até que bacias atribuídas a diferentes marcadores se encontrem nas linhas watershed. Em muitos casos, os marcadores são escolhidos como mínimos locais da imagem, a partir dos quais as bacias são inundadas. Uma boa explicação da Transformada Watersher é fornecida na página do Centro de Morfologia Matemática do o laboratório de processamento de imagens do MINES ParisTech: http://www.cmm.mines-paristech.fr/~beucher/wtshed.html. Theanimation below is from there:

![](http://www.cmm.mines-paristech.fr/~beucher/lpe1.gif)

### Implementação do Watershed com SciKit

No exemplo abaixo, dois círculos sobrepostos devem ser separados. Para fazer isso, calcula-se uma imagem que é a distância ao fundo. Os máximos desta distância (ou seja, os mínimos do oposto da distância) são escolhidos como marcadores, e a inundação das bacias a partir de tais marcadores separa os dois círculos ao longo de uma linha divisória.

In [ ]:
import numpy as np
from ipywidgets import interact, interactive, interact_manual
import cv2, scipy
from skimage import data
from skimage.color import rgb2gray
from skimage.filters import sobel
from skimage.feature import canny
from skimage.segmentation import mark_boundaries
from matplotlib import pyplot as plt
from scipy import ndimage as ndi

from skimage.segmentation import felzenszwalb, slic, quickshift, watershed

#from skimage.morphology import watershed
from skimage.feature import peak_local_max


image = cv2.imread("ct-02.jpg", cv2.IMREAD_GRAYSCALE)
# shifted = cv2.pyrMeanShiftFiltering(image, 21, 51)

gradient_methods = [
    'None',
    'Sobel',
    'Canny'
]

def my_watershed(gradient_method, markers, colormap):
    global image
    colormap = eval('plt.cm.' + colormap)

    if (gradient_method == 'None'):
        input_image = image
    elif (gradient_method == 'Sobel'):
        input_image = sobel(image)
    else:
        input_image = canny(image)

    # Gera os marcadores como máximos locais da distância ao fundo
    # Aplica transformação de distância euclidiana exata
    distance = ndi.distance_transform_edt(image)
    # A função peak_local_max retorna as coordenadas dos picos locais (máximos) em uma imagem.
    # Um filtro máximo é usado para encontrar máximos locais. Esta operação dilata o original
    # imagem e mescla máximos locais vizinhos mais próximos do que o tamanho da dilatação.
    # Coordenadas onde a imagem original é igual à imagem dilatada são retornadas como máximos locais.
    local_maxi = peak_local_max(distance, footprint=np.ones((8,8)), labels=image, min_distance=20)
    # Pegue o número...
    #markers = ndi.label(local_maxi)[0]
    #labels = watershed(-distance, markers, mask=image)
    labels = watershed(input_image, markers=markers, connectivity=1, compactness=0.0, watershed_line=True)

    fig, axes = plt.subplots(ncols=3, figsize=(20, 7), sharex=True, sharey=True)
    ax = axes.ravel()

    ax[0].imshow((mark_boundaries(image, labels)), cmap=plt.cm.gray)
    ax[0].set_title('Original with Boundaries')
    if (gradient_method == 'None'):
        #ax[1].imshow(distance, cmap=plt.cm.gray, interpolation='nearest')
        ax[1].imshow(distance, cmap=plt.cm.gray)
        ax[1].set_title('Distances')
        # Plot the local maxima...
        ax[1].autoscale(False)
        ax[1].plot(local_maxi[:, 1], local_maxi[:, 0], 'r.')
    else:
        ax[1].imshow(input_image, cmap=plt.cm.gray)
        ax[1].set_title(gradient_method)

    ax[2].imshow(labels, cmap=colormap, interpolation='nearest')
    ax[2].set_title('Regions')

    for a in ax:
        a.set_axis_off()

    fig.tight_layout()
    plt.show()

interact_manual(my_watershed, gradient_method=gradient_methods, markers = (0, 100), colormap = ['nipy_spectral', 'hot', 'magma', 'seismic'])

interactive(children=(Dropdown(description='gradient_method', options=('None', 'Sobel', 'Canny'), value='None'…

<function __main__.my_watershed(gradient_method, markers, colormap)>

# Créditos

* A seção Watershed foi parcialmente baseada em material disponível em:
  * https://www.pyimagesearch.com/2015/11/02/watershed-opencv/
  * https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_watershed.html
  * https://docs.opencv.org/3.4.3/d3/db4/tutorial_py_watershed.html
* A seção Mumford & Shah foi parcialmente baseada em material disponível em:
  * A excelente e inovadora implementação Python da variante Ambrosio-Tortorelli-Minimizer do M&S em: https://github.com/jacobgil/Ambrosio-Tortorelli-Minimizer
  * A documentação da variante Ambrosio-Tortorelli-Minimizer do M&S em: https://jacobgil.github.io/computervision/mumford-shah and http://jacobcv.blogspot.com/2015/04/smoothing-images-with-mumford-shah.html
* Truques gerais para exibir imagens vieram daqui: https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_gui/py_image_display/py_image_display.html
* Também usamos algumas dicas gerais de: https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_segmentations.html#sphx-glr-auto-examples-segmentation-plot-segmentations-py
* Também usamos material do Professor Aldo von Wangenheim da UFSC